In [1]:
import pandas as pd
import numpy as np
import util
import yaml
import tensorflow as tf
from trace_plotting import plot_two_traces
import os
import time

os.environ["CUDA_VISIBLE_DEVICES"]="1" #model will be trained on GPU 1
data_path = 'data'

In [2]:
# Load data
train = np.load(data_path + '/ae_training_1mil.npy').astype(np.float32)
own_set = np.load(data_path + '/own_set_1mil.npy').astype(np.float32)
other_set = np.load(data_path + '/other_set_1mil.npy').astype(np.float32)

# Test Set is used to validate AE on
test =other_set 

In [3]:

#data = np.load(data_path + '/traces_266517.npy')
#train_size = data.shape[0] - int(data.shape[0]/3)
#train = data[:train_size].astype(np.float32)
#test = data[train_size:].astype(np.float32)


In [4]:
print("Train size: " + str(train.shape[0]))
print("own_set size: " + str(own_set.shape[0]))
print("other_set size: " + str(other_set.shape[0]))




Train size: 2566
own_set size: 517
other_set size: 2161


In [5]:
# Definition of Model
class AE(tf.keras.Model):
    def __init__(self, batch_size, timesteps, feats_size, latent_dim=50):
        super(AE, self).__init__()
        self.latent_dim = latent_dim

        self._batch_size = batch_size
        self._timesteps  = timesteps
        self._feats_size = feats_size

        self.encoder = tf.keras.Sequential(
            [
                # Input data shape: (batch_size, 256, 3)
                tf.keras.layers.InputLayer(input_shape=(self._timesteps, self._feats_size)),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(units = self._timesteps * self._feats_size, activation=tf.nn.relu),
                #tf.keras.layers.Dense(units= self._timesteps * self._feats_size, activation=tf.nn.relu),
                # No activation
                tf.keras.layers.Dense(latent_dim),
                
            ]
        )

        self.decoder = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
                tf.keras.layers.Dense(units=self._timesteps * self._feats_size, activation=tf.nn.relu),
                tf.keras.layers.Dense(units=self._timesteps * self._feats_size, activation=None),
                tf.keras.layers.Reshape(target_shape=(self._timesteps, self._feats_size)),
            ]
        )


    def run_model(self, x):
        s = self.encoder(x)
        x_recon = self.decoder(s)
        return s, x_recon

    def compute_loss(self, x):
        s, x_recon = self.run_model(x)
        # Simple Squered Diff
        cost = tf.math.reduce_mean(tf.square(tf.subtract(x_recon, x)))
        return cost
    
    #Training and Test Step
    @tf.function
    def train_step(self, x, optimizer):
        """Executes one training step and returns the loss.

        This function computes the loss and gradients, and uses the latter to
        update the model's parameters.
        """
        with tf.GradientTape() as tape:
            loss = self.compute_loss(x)
        gradients = tape.gradient(loss, self.trainable_variables)
        optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return loss

    @tf.function
    def test_step(self, x):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).

        t_loss =  self.compute_loss(x)
        return t_loss

In [6]:

# Train Model

batch_size = 256
timesteps = train.shape[1]
feats_len = train.shape[2]

# Create an instance of the model
model = AE(batch_size, timesteps, feats_len)
optimizer = tf.keras.optimizers.Adam(1e-4)

# Use tf.data for Data handling 
batch_size = 256
dataset = tf.data.Dataset.from_tensor_slices(train)
dataset = dataset.shuffle(len(train), reshuffle_each_iteration=True)
# Create Batches from dataset
# Drop remainder: Drops the last Batch, as it has not the same shape as the other batches
#dataset  = dataset.batch(batch_size, drop_remainder  = True)
dataset  = dataset.batch(batch_size)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)



EPOCHS = 1000
num_samples = len(train)
start = time.time()
for epoch in range(EPOCHS):
    
    next_batch_start = 0
    idx_end = 0
    # Iterate Batches
    for batch in dataset:
        # Train step with optimization
        loss = model.train_step(batch, optimizer)

    test_loss = model.test_step(test)

    template = 'Epoch {:3d}, Loss: {:.4f}, Test Loss: {:.4f}'
    print(template.format(epoch + 1,loss.numpy(),test_loss.numpy()))

    # s, x_recon = model.run_model(test)
    # print(test[0, 0:10])
    # print(s[0, 0:10])
    # print(x_recon[0, 0:10])
    
end = time.time()
print("Training duration: {}".format(end - start))
s, x_recon = model.run_model(test)


print(test[0, 0:10])
print(s[0, 0:10].numpy())
print(x_recon[0, 0:10].numpy())

Epoch   1, Loss: 758.9424, Test Loss: 734.8417
Epoch   2, Loss: 410.5894, Test Loss: 378.4577
Epoch   3, Loss: 142.5995, Test Loss: 128.0090
Epoch   4, Loss: 35.2017, Test Loss: 30.9736
Epoch   5, Loss: 7.6767, Test Loss: 6.8123
Epoch   6, Loss: 2.3731, Test Loss: 2.1658
Epoch   7, Loss: 1.3522, Test Loss: 1.1060
Epoch   8, Loss: 0.5597, Test Loss: 0.5860
Epoch   9, Loss: 0.4397, Test Loss: 0.4172
Epoch  10, Loss: 0.5437, Test Loss: 0.3717
Epoch  11, Loss: 0.5844, Test Loss: 0.3547
Epoch  12, Loss: 0.2312, Test Loss: 0.3509
Epoch  13, Loss: 0.5046, Test Loss: 0.3473
Epoch  14, Loss: 0.1190, Test Loss: 0.3444
Epoch  15, Loss: 0.2751, Test Loss: 0.3427
Epoch  16, Loss: 0.3757, Test Loss: 0.3416
Epoch  17, Loss: 0.5677, Test Loss: 0.3408
Epoch  18, Loss: 0.4081, Test Loss: 0.3397
Epoch  19, Loss: 0.3739, Test Loss: 0.3398
Epoch  20, Loss: 0.3792, Test Loss: 0.3384
Epoch  21, Loss: 0.3226, Test Loss: 0.3375
Epoch  22, Loss: 0.5519, Test Loss: 0.3369
Epoch  23, Loss: 0.4680, Test Loss: 0.33

Epoch 192, Loss: 0.0674, Test Loss: 0.0563
Epoch 193, Loss: 0.0565, Test Loss: 0.0539
Epoch 194, Loss: 0.0477, Test Loss: 0.0532
Epoch 195, Loss: 0.0933, Test Loss: 0.0533
Epoch 196, Loss: 0.0512, Test Loss: 0.0529
Epoch 197, Loss: 0.0495, Test Loss: 0.0514
Epoch 198, Loss: 0.0540, Test Loss: 0.0534
Epoch 199, Loss: 0.0378, Test Loss: 0.0508
Epoch 200, Loss: 0.0697, Test Loss: 0.0526
Epoch 201, Loss: 0.0286, Test Loss: 0.0522
Epoch 202, Loss: 0.0247, Test Loss: 0.0499
Epoch 203, Loss: 0.0371, Test Loss: 0.0499
Epoch 204, Loss: 0.0303, Test Loss: 0.0496
Epoch 205, Loss: 0.0814, Test Loss: 0.0505
Epoch 206, Loss: 0.0731, Test Loss: 0.0519
Epoch 207, Loss: 0.0524, Test Loss: 0.0504
Epoch 208, Loss: 0.0222, Test Loss: 0.0495
Epoch 209, Loss: 0.0465, Test Loss: 0.0500
Epoch 210, Loss: 0.0428, Test Loss: 0.0549
Epoch 211, Loss: 0.0502, Test Loss: 0.0497
Epoch 212, Loss: 0.0256, Test Loss: 0.0482
Epoch 213, Loss: 0.0567, Test Loss: 0.0512
Epoch 214, Loss: 0.1020, Test Loss: 0.0506
Epoch 215, 

Epoch 386, Loss: 0.0156, Test Loss: 0.0376
Epoch 387, Loss: 0.0315, Test Loss: 0.0394
Epoch 388, Loss: 0.0372, Test Loss: 0.0367
Epoch 389, Loss: 0.0233, Test Loss: 0.0383
Epoch 390, Loss: 0.0147, Test Loss: 0.0368
Epoch 391, Loss: 0.0311, Test Loss: 0.0356
Epoch 392, Loss: 0.0796, Test Loss: 0.0474
Epoch 393, Loss: 0.0388, Test Loss: 0.0459
Epoch 394, Loss: 0.0471, Test Loss: 0.0489
Epoch 395, Loss: 0.0247, Test Loss: 0.0377
Epoch 396, Loss: 0.0307, Test Loss: 0.0417
Epoch 397, Loss: 0.0169, Test Loss: 0.0373
Epoch 398, Loss: 0.0441, Test Loss: 0.0399
Epoch 399, Loss: 0.0268, Test Loss: 0.0364
Epoch 400, Loss: 0.0443, Test Loss: 0.0361
Epoch 401, Loss: 0.0343, Test Loss: 0.0362
Epoch 402, Loss: 0.0248, Test Loss: 0.0358
Epoch 403, Loss: 0.0693, Test Loss: 0.0396
Epoch 404, Loss: 0.0741, Test Loss: 0.0408
Epoch 405, Loss: 0.0104, Test Loss: 0.0369
Epoch 406, Loss: 0.0410, Test Loss: 0.0367
Epoch 407, Loss: 0.0311, Test Loss: 0.0433
Epoch 408, Loss: 0.0532, Test Loss: 0.0480
Epoch 409, 

Epoch 582, Loss: 0.0718, Test Loss: 0.0374
Epoch 583, Loss: 0.0181, Test Loss: 0.0333
Epoch 584, Loss: 0.0621, Test Loss: 0.0404
Epoch 585, Loss: 0.0535, Test Loss: 0.0412
Epoch 586, Loss: 0.0265, Test Loss: 0.0365
Epoch 587, Loss: 0.0181, Test Loss: 0.0305
Epoch 588, Loss: 0.0269, Test Loss: 0.0296
Epoch 589, Loss: 0.0258, Test Loss: 0.0352
Epoch 590, Loss: 0.0768, Test Loss: 0.0435
Epoch 591, Loss: 0.0158, Test Loss: 0.0327
Epoch 592, Loss: 0.0602, Test Loss: 0.0347
Epoch 593, Loss: 0.0735, Test Loss: 0.0466
Epoch 594, Loss: 0.0145, Test Loss: 0.0343
Epoch 595, Loss: 0.0180, Test Loss: 0.0295
Epoch 596, Loss: 0.0175, Test Loss: 0.0297
Epoch 597, Loss: 0.0339, Test Loss: 0.0398
Epoch 598, Loss: 0.0274, Test Loss: 0.0340
Epoch 599, Loss: 0.0339, Test Loss: 0.0293
Epoch 600, Loss: 0.0278, Test Loss: 0.0283
Epoch 601, Loss: 0.0444, Test Loss: 0.0345
Epoch 602, Loss: 0.0299, Test Loss: 0.0402
Epoch 603, Loss: 0.0173, Test Loss: 0.0296
Epoch 604, Loss: 0.0203, Test Loss: 0.0321
Epoch 605, 

Epoch 778, Loss: 0.0269, Test Loss: 0.0285
Epoch 779, Loss: 0.0112, Test Loss: 0.0242
Epoch 780, Loss: 0.0109, Test Loss: 0.0216
Epoch 781, Loss: 0.0231, Test Loss: 0.0273
Epoch 782, Loss: 0.0244, Test Loss: 0.0236
Epoch 783, Loss: 0.0125, Test Loss: 0.0271
Epoch 784, Loss: 0.0294, Test Loss: 0.0362
Epoch 785, Loss: 0.0313, Test Loss: 0.0307
Epoch 786, Loss: 0.0217, Test Loss: 0.0246
Epoch 787, Loss: 0.0141, Test Loss: 0.0221
Epoch 788, Loss: 0.0513, Test Loss: 0.0297
Epoch 789, Loss: 0.0175, Test Loss: 0.0233
Epoch 790, Loss: 0.0330, Test Loss: 0.0284
Epoch 791, Loss: 0.0188, Test Loss: 0.0267
Epoch 792, Loss: 0.0236, Test Loss: 0.0351
Epoch 793, Loss: 0.0147, Test Loss: 0.0228
Epoch 794, Loss: 0.0276, Test Loss: 0.0281
Epoch 795, Loss: 0.0123, Test Loss: 0.0238
Epoch 796, Loss: 0.0088, Test Loss: 0.0225
Epoch 797, Loss: 0.0110, Test Loss: 0.0238
Epoch 798, Loss: 0.0141, Test Loss: 0.0237
Epoch 799, Loss: 0.0298, Test Loss: 0.0268
Epoch 800, Loss: 0.0215, Test Loss: 0.0357
Epoch 801, 

Epoch 972, Loss: 0.0115, Test Loss: 0.0293
Epoch 973, Loss: 0.0327, Test Loss: 0.0276
Epoch 974, Loss: 0.0225, Test Loss: 0.0251
Epoch 975, Loss: 0.0075, Test Loss: 0.0192
Epoch 976, Loss: 0.0273, Test Loss: 0.0228
Epoch 977, Loss: 0.0074, Test Loss: 0.0193
Epoch 978, Loss: 0.0196, Test Loss: 0.0268
Epoch 979, Loss: 0.0272, Test Loss: 0.0285
Epoch 980, Loss: 0.0171, Test Loss: 0.0384
Epoch 981, Loss: 0.0135, Test Loss: 0.0220
Epoch 982, Loss: 0.0084, Test Loss: 0.0199
Epoch 983, Loss: 0.0099, Test Loss: 0.0196
Epoch 984, Loss: 0.0677, Test Loss: 0.0418
Epoch 985, Loss: 0.0460, Test Loss: 0.0331
Epoch 986, Loss: 0.0304, Test Loss: 0.0315
Epoch 987, Loss: 0.0135, Test Loss: 0.0225
Epoch 988, Loss: 0.0127, Test Loss: 0.0191
Epoch 989, Loss: 0.0088, Test Loss: 0.0207
Epoch 990, Loss: 0.0125, Test Loss: 0.0220
Epoch 991, Loss: 0.0148, Test Loss: 0.0218
Epoch 992, Loss: 0.0123, Test Loss: 0.0246
Epoch 993, Loss: 0.0130, Test Loss: 0.0232
Epoch 994, Loss: 0.0182, Test Loss: 0.0232
Epoch 995, 

In [7]:
from itertools import cycle

# Get Traces as lists
*t_orig, = test
*t_recon, = x_recon


# Get Iterator to cycle through
t_orig_cycle = cycle(t_orig)
t_recon_cycle = cycle(t_recon)



In [8]:
# Cycle and print traces
t1 = next(t_orig_cycle)
t2 = next(t_recon_cycle)

# Print Original Trace (red) and Reconstructed Trace (blue)
plot_two_traces(t1, t2)

# Test Trace Threads #

- Rounding Thread
- Signal Noise
- White Noise


In [9]:
# Test threads

from threats import *
from trace_plotting import plot_two_traces

first_trace = test[0]

In [10]:
# Rounding Thread
rounded_trace = remove_non_significant_bits(first_trace, spatial_decimals=1)
plot_two_traces(first_trace, rounded_trace)

In [11]:
# Added Signal Noise
noisy_trace = add_signal_noise(first_trace, signal_to_noise_ratio=40)
plot_two_traces(first_trace, noisy_trace)

In [12]:
# Added White Noise
noisy_trace2 = add_white_noise(first_trace, spatial_variance=.05)
plot_two_traces(first_trace, noisy_trace2)

In [13]:
# Remove Points
pts_removed = remove_random_points(first_trace, removal_percentage=.05)
plot_two_traces(first_trace, pts_removed)

In [14]:
# Remove Points
pts_removed = replace_non_skeleton_points_with_start_point(first_trace, epsilon=.005)
plot_two_traces(first_trace, pts_removed)

In [15]:
# Remove Points
pts_removed = replace_non_skeleton_points_with_path(first_trace, epsilon=.005)
plot_two_traces(first_trace, pts_removed)

# Traces Identification #


Now we have a trained Autoencoder, which can produce latent representations s.
To identfy Traces, we produce latent representations for all the traces, which we want to be able to identify.

Then experiment: 
    - for a trace t how close is s(t) to s(a(t)), where a represents an attack
    - how far away is a different trace 



In [16]:


# Get the representations of own set
s_own_set, rec_own_set = model.run_model(own_set)
s_own_set = s_own_set.numpy()


In [17]:
# Use Attacks on own_set



# 1. Attack
attack1 =[]
for t in own_set:
    attack1.append(remove_non_significant_bits(t, spatial_decimals=1))
attack1 = np.stack( attack1, axis=0 )    
    
# 2. Attack
attack2 = []
for t in own_set:
    attack2.append(add_signal_noise(t, signal_to_noise_ratio=40))
attack2 = np.stack( attack2, axis=0 ) 
    
# 3. Attack
attack3 = []
for t in own_set:
    attack3.append(add_white_noise(t, spatial_variance=.05))
attack3 = np.stack( attack3, axis=0 )

# 4. Attack
attack4 = []
for t in own_set:
    attack4.append(remove_random_points(t, removal_percentage=.05))
attack4 = np.stack( attack4, axis=0 )

# 5. Attack
attack5 = []
for t in own_set:
    attack5.append(replace_non_skeleton_points_with_start_point(t, epsilon=.005))
attack5 = np.stack( attack5, axis=0 )

# 6. Attack
attack6 = []
for t in own_set:
    attack6.append(replace_non_skeleton_points_with_path(t, epsilon=.005))
attack6 = np.stack( attack6, axis=0 )

In [18]:
other_set.shape

(2161, 256, 3)

In [19]:
own_set.shape

(517, 256, 3)

In [20]:
# Combine with other_set
mixed_set = np.concatenate((other_set, attack1, attack2, attack3, attack4, attack5, attack6), axis=0)

In [21]:
# Create Label Vector
label_own = np.ones(attack1.shape[0] * 6, dtype=bool)
label_others = np.zeros(other_set.shape[0], dtype=bool)
labels = np.concatenate((label_others, label_own), axis=0)

In [22]:
def compare_traces(known_trace_encodings, trace_encoding_to_check, tolerance=0.4):
    """
    Compare a list of face encodings against a candidate encoding to see if they match.
    :param known_face_encodings: A list of known face encodings
    :param face_encoding_to_check: A single face encoding to compare against the list
    :param tolerance: How much distance between traces to consider it a match. Lower is more strict. 0.4 is typical best performance.
    :return: A list of True/False values indicating which known_trace_encodings match the trace encoding to check
    """
    ret = True in list(trace_distance(known_trace_encodings, trace_encoding_to_check) <= tolerance)
    return ret


def trace_distance(trace_encodings, trace_to_compare):
    """
    Given a list of encodings, compare them to a known encoding and get a euclidean distance
    for each comparison. The distance tells you how similar the encodings are.
    :param trace_encodings: List of encodings to compare
    :param trace_to_compare: A trace encoding to compare against
    :return: A numpy ndarray with the distance for each trace in the same order as the 'traces' array
    """
    if len(trace_encodings) == 0:
        return np.empty((0))
    
    return np.linalg.norm(trace_encodings - trace_to_compare, axis=1)

In [23]:
# Run through Test set and get predictions
pred_list = []
for trace in mixed_set:    
    s, _ = model.run_model(np.expand_dims(trace, axis=0))
    prediction = compare_traces(s_own_set, s)
    
    pred_list.append(prediction)
    
predictions = np.asarray(pred_list)

In [24]:
# Evaluate Predictions
from sklearn.metrics import accuracy_score, f1_score
accuracy = accuracy_score(labels,predictions)
f1 = f1_score(labels, predictions)
print("Accuracy: {:.3f}".format(accuracy))
print("F1 Score: {:.3f}".format(f1))

Accuracy: 0.971
F1 Score: 0.975


## Rounding Thread

In [25]:
other_set_small = other_set[:len(own_set)]

In [26]:
# Combine with other_set
mixed_set = np.concatenate((other_set_small, attack1), axis=0)

In [27]:
# Create Label Vector
label_own = np.ones(attack1.shape[0], dtype=bool)
label_others = np.zeros(other_set_small.shape[0], dtype=bool)
labels = np.concatenate((label_others, label_own), axis=0)

In [28]:
# Run through Test set and get predictions
pred_list = []
for trace in mixed_set:    
    s, _ = model.run_model(np.expand_dims(trace, axis=0))
    prediction = compare_traces(s_own_set, s)
    
    pred_list.append(prediction)
    
predictions = np.asarray(pred_list)

In [29]:
# Evaluate Predictions
from sklearn.metrics import accuracy_score, f1_score
accuracy = accuracy_score(labels,predictions)
f1 = f1_score(labels, predictions)
print("Accuracy: {:.3f}".format(accuracy))
print("F1 Score: {:.3f}".format(f1))

Accuracy: 0.974
F1 Score: 0.973


## Signal Noise

In [30]:
# Combine with other_set
mixed_set = np.concatenate((other_set_small, attack2), axis=0)

In [31]:
# Run through Test set and get predictions
pred_list = []
for trace in mixed_set:    
    s, _ = model.run_model(np.expand_dims(trace, axis=0))
    prediction = compare_traces(s_own_set, s)
    
    pred_list.append(prediction)
    
predictions = np.asarray(pred_list)

In [32]:
# Evaluate Predictions
from sklearn.metrics import accuracy_score, f1_score
accuracy = accuracy_score(labels,predictions)
f1 = f1_score(labels, predictions)
print("Accuracy: {:.3f}".format(accuracy))
print("F1 Score: {:.3f}".format(f1))

Accuracy: 0.999
F1 Score: 0.999


## White Noise

In [33]:
# Combine with other_set
mixed_set = np.concatenate((other_set_small, attack3), axis=0)

In [34]:
# Run through Test set and get predictions
pred_list = []
for trace in mixed_set:    
    s, _ = model.run_model(np.expand_dims(trace, axis=0))
    prediction = compare_traces(s_own_set, s)
    
    pred_list.append(prediction)
    
predictions = np.asarray(pred_list)

In [35]:
# Evaluate Predictions
from sklearn.metrics import accuracy_score, f1_score
accuracy = accuracy_score(labels,predictions)
f1 = f1_score(labels, predictions)
print("Accuracy: {:.3f}".format(accuracy))
print("F1 Score: {:.3f}".format(f1))

Accuracy: 0.999
F1 Score: 0.999


## Remove random points

In [36]:
# Combine with other_set
mixed_set = np.concatenate((other_set_small, attack4), axis=0)

In [37]:
# Run through Test set and get predictions
pred_list = []
for trace in mixed_set:    
    s, _ = model.run_model(np.expand_dims(trace, axis=0))
    prediction = compare_traces(s_own_set, s)
    
    pred_list.append(prediction)
    
predictions = np.asarray(pred_list)

In [38]:
# Evaluate Predictions
from sklearn.metrics import accuracy_score, f1_score
accuracy = accuracy_score(labels,predictions)
f1 = f1_score(labels, predictions)
print("Accuracy: {:.3f}".format(accuracy))
print("F1 Score: {:.3f}".format(f1))

Accuracy: 0.996
F1 Score: 0.996


## Replace non skeleton points with start point

In [39]:
# Combine with other_set
mixed_set = np.concatenate((other_set_small, attack5), axis=0)

In [40]:
# Run through Test set and get predictions
pred_list = []
for trace in mixed_set:    
    s, _ = model.run_model(np.expand_dims(trace, axis=0))
    prediction = compare_traces(s_own_set, s)
    
    pred_list.append(prediction)
    
predictions = np.asarray(pred_list)

In [41]:
# Evaluate Predictions
from sklearn.metrics import accuracy_score, f1_score
accuracy = accuracy_score(labels,predictions)
f1 = f1_score(labels, predictions)
print("Accuracy: {:.3f}".format(accuracy))
print("F1 Score: {:.3f}".format(f1))

Accuracy: 0.882
F1 Score: 0.867


## Replace non skeleton points with path

In [42]:
# Combine with other_set
mixed_set = np.concatenate((other_set_small, attack6), axis=0)

In [43]:
# Run through Test set and get predictions
pred_list = []
for trace in mixed_set:    
    s, _ = model.run_model(np.expand_dims(trace, axis=0))
    prediction = compare_traces(s_own_set, s)
    
    pred_list.append(prediction)
    
predictions = np.asarray(pred_list)

In [44]:
# Evaluate Predictions
from sklearn.metrics import accuracy_score, f1_score
accuracy = accuracy_score(labels,predictions)
f1 = f1_score(labels, predictions)
print("Accuracy: {:.3f}".format(accuracy))
print("F1 Score: {:.3f}".format(f1))

Accuracy: 0.998
F1 Score: 0.998
